# Cliff Walking

# Task 01. SRASA from scratch
1. SARSA방식으로 Q-table을 완성하자.
2. 완성된 Q-table과 env.render()를 이용하여 어떻게 답을 찾는지 관찰하자.

# Task 02. Q-Learning from scratch
1. Q-learning방식으로 Q-table을 완성하자.
2. 완성된 Q-table과 env.render()를 이용하여 어떻게 답을 찾는지 관찰하자.

### CliffWalking-v0
<img src='https://miro.medium.com/max/1184/1*FBJfEd_cuVW1CsDK7gCQsg.png'>

In [1]:
import numpy as np
import gym

In [2]:
env = gym.make('CliffWalking-v0')

# I. SARSA

### 1. parameter들

In [3]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

### 2. SARSA!

In [4]:
print(env.observation_space.n, env.action_space.n)

48 4


In [5]:
Q = np.zeros([env.observation_space.n,
              env.action_space.n])

rewards_sarsa = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False
    a0 = env.action_space.sample() # 처음엔 랜덤무브

    while True :
        s1, r1, done, _ = env.step(a0) # 일단 움직인다.

        if np.random.uniform() < epsilon :  # 무슨 뜻? 15% 확률로 램덤하게 액션 선택
            a1 = env.action_space.sample()
        else : 
            greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
            a1 = np.random.choice(greedy_actions) # 무슨 뜻? 해당 state에서 제일 좋은 action을 선택

        # Update 한다! 매 action 마다
        # ( s0, a0, r1, s1, a1 ) 로 부터!
        # (참고) Monte Carlo에서는 전체 episode 끝난후에 value 업데이트함.
        
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*Q[s1, a1] - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_sarsa.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state
        a0 = a1 # 다음 루프에선 이것이 직전 action

    if (i+1) % 8000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        for rows in Q.reshape([-1,4,4]):
            print(*rows)

===========  에피소드 : 8000  ============
[-18.51125972 -17.2446845  -20.45852334 -18.3271459 ] [-17.22873298 -15.96170831 -18.07535398 -18.30347188] [-16.22481494 -14.66531789 -19.21905902 -18.10233545] [-14.44164308 -13.2539626  -17.17682518 -16.22055272]
[-13.37189404 -11.78776384 -15.36822123 -14.75396644] [-12.30633885 -10.42533436 -13.10129298 -13.59502856] [-10.85610574  -9.48432901 -12.72083892 -12.18086638] [ -9.55029541  -8.46758522 -11.80208099 -11.51779846]
[ -8.12011798  -7.3565452  -10.69359065  -9.78768644] [-7.05042857 -6.56440466 -7.95731714 -8.64058328] [-6.26334791 -5.10333751 -7.635075   -7.46555921] [-4.81092477 -4.8510729  -3.80238688 -6.5417768 ]
[-18.46879202 -18.96149355 -24.34451581 -19.22968641] [-17.13487104 -18.7892613  -30.98671222 -19.96377766] [-16.95153567 -20.26456585 -23.48315121 -22.32352952] [-14.3705027  -15.94976695 -26.25012579 -17.99669799]
[-13.21006622 -14.34391085 -19.88656143 -16.21572846] [-13.18719066 -12.36667282 -16.48813523 -15.13417762] [

### 학습 완료된 것 구경
### 무한 루프 주의
위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.<br>
> <font color=red>SARSA 경우 Cliff를 멀찍히 떨어져서 목적지로 감. (최적의 경로는 아님)</font>

In [6]:
s0 = env.reset()
done = False
a0 = env.action_space.sample() # 처음엔 랜덤무브

while True :
    env.render()
    s1, r1, done, _ = env.step(a0) # 일단 움직인다.

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻? Greed 방법만으로 최적의 action만 수행

    # 무한 루프 걸린다면 위의 두줄 주석, 아래를 주석 해제
    # if np.random.uniform() < epsilon :  # 무슨 뜻?
    #     a1 = env.action_space.sample()
    # else : 
    #     greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    #     a1 = np.random.choice(greedy_actions) # 무슨 뜻?


    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

x  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  x  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  x  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  x  o

# II. Q-Learning


### 1. parameter들

In [7]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

### 2. Q-learning

In [8]:
print(env.observation_space.n, env.action_space.n)

48 4


In [9]:
Q = np.zeros([env.observation_space.n,
              env.action_space.n])

rewards_ql = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False

    while True :
        # 행동을 선택
        if np.random.uniform() < epsilon :  # 무슨 뜻? 15% 확률로 random action 수행
            a0 = env.action_space.sample()
        else : 
            greedy_actions = np.argwhere(Q[s0, :] == np.amax(Q[s0, :])).reshape(-1)
            a0 = np.random.choice(greedy_actions) # 무슨 뜻? 해당 state에서 제일 좋은 action을 선택

        # a0 를 이용하여, 환경과 상호작용하고, s1, r1, done, _를 선언하시오.
        s1, r1, done, _ = env.step(a0)
        
        # ( s0, a0, r1, s1 ) 로 부터! 차이점 주의!
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*np.max(Q[s1,:]) - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_ql.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state

    if (i+1) % 8000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        for rows in Q.reshape([-1,4,4]):
            print(*rows)

===========  에피소드 : 8000  ============
[-13.75544859 -13.69718272 -13.69253347 -13.95666124] [-13.05987791 -12.88341253 -12.88058771 -13.02827996] [-12.23599597 -11.92271056 -11.9240998  -12.75161875] [-11.2933166  -10.94072976 -10.94126083 -11.88670012]
[-10.10771137  -9.95267877  -9.95271461 -11.39327897] [ -9.53280293  -8.96262287  -8.96282047 -10.39182495] [-8.61277092 -7.97150199 -7.97154744 -8.94542909] [-7.39850863 -6.97881607 -6.9788284  -8.07544481]
[-6.24736835 -5.98493714 -5.98493134 -7.47584372] [-5.6374349  -4.98997231 -4.98996872 -6.19631551] [-4.60519843 -3.99399478 -3.99399496 -5.22201904] [-3.65478497 -3.5329585  -2.997001   -3.98488501]
[-14.49561386 -12.92228529 -12.92228529 -13.89753718] [-13.80652235 -11.93421951 -11.93421951 -13.90201281] [-12.88136162 -10.94516467 -10.94516467 -12.92150738] [-11.9214002   -9.95511979  -9.95511979 -11.93346419]
[-10.93544587  -8.96408387  -8.96408387 -10.94479358] [-9.94332285 -7.97205593 -7.97205593 -9.95483332] [-8.95562798 -6.9

### 학습 완료된 것 구경
### 무한 루프 주의
위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.<br>
> <font color=red>Q-learning 경우 Cliff를 붙어서 목적지로 감. (최적의 경로임)</font>

In [10]:
s0 = env.reset()
done = False
a0 = env.action_space.sample() # 처음엔 랜덤무브

while True :
    env.render()
    s1, r1, done, _ = env.step(a0) # 일단 움직인다.

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻?

    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  o  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  x  o  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  x  o  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  x  o  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  x  o  o  o  o  o  o  o
o  C  C  C  C  C  C  C  C  C  C  T

o  o  o  o  o